In [1]:
from utils.spark_session import get_spark_session
from pyspark.sql import SparkSession

spark: SparkSession = get_spark_session()

25/10/12 16:27:11 WARN Utils: Your hostname, DESKTOP-9VM3SA1 resolves to a loopback address: 127.0.1.1; using 172.28.82.250 instead (on interface eth0)
25/10/12 16:27:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/dottier/venvs/flights_env/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/dottier/.ivy2/cache
The jars for the packages stored in: /home/dottier/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c634433c-39ac-446e-988a-5764e29ce017;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 164ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   

In [2]:
spark.sql("DROP TABLE IF EXISTS bronze.flights")

25/10/01 10:23:32 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/10/01 10:23:32 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/10/01 10:23:33 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/10/01 10:23:33 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore dottier@127.0.1.1
25/10/01 10:23:33 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[]

In [4]:
spark.sql("SHOW TABLES IN bronze").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|   bronze| airports|      false|
|   bronze|  flights|      false|
+---------+---------+-----------+



In [5]:
spark.sql("SHOW TABLES IN silver").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|   silver|         dim_players|      false|
|   silver|          dim_stages|      false|
|   silver|           dim_teams|      false|
|   silver|   fct_match_summary|      false|
|   silver|fct_player_match_...|      false|
+---------+--------------------+-----------+



In [6]:
spark.sql("SHOW TABLES IN gold").show()

25/10/12 16:28:05 WARN ObjectStore: Failed to get database gold, returning NoSuchObjectException


AnalysisException: [SCHEMA_NOT_FOUND] The schema `gold` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a catalog, verify the current_schema() output, or qualify the name with the correct catalog.
To tolerate the error on drop use DROP SCHEMA IF EXISTS.

In [2]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, to_timestamp, regexp_extract, coalesce

def transform_timestamps(df: DataFrame) -> DataFrame:
    source_format = "yyyy-MM-dd HH:mm'Z'"
    iso_8601_format = "yyyy-MM-dd'T'HH:mm:ss'Z'"
    local_with_offset_format = "yyyy-MM-dd HH:mmXXX"

    utc_cols = [c for c in df.columns if c.endswith("_utc")]
    local_cols = [c for c in df.columns if c.endswith("_local")]

    for utc_col in utc_cols:
        df = df.withColumn(
            utc_col,
            coalesce(
                to_timestamp(col(utc_col), source_format),
                to_timestamp(col(utc_col), iso_8601_format),
            )
        )

    for local_col in local_cols:
        df = df.withColumn(
            local_col + "_ntz",
            to_timestamp(col(local_col), local_with_offset_format)
        )

    return df

In [22]:
read_options = {
    "where": f"ingestion_hour = '2025-10-02-12'",
}
bronze_flights = read_df(spark, "bronze", "flights", **read_options)

bronze_flights.count()

Successfully read bronze.flights


30

In [23]:
from utils.io_utils import read_df

bronze_flights = read_df(spark, "bronze", "flights")
bronze_airports = read_df(spark, "bronze", "airports")

Successfully read bronze.flights
Successfully read bronze.airports


In [4]:
import logging, re
import pandas as pd
from pyspark.sql.functions import pandas_udf, col
from pyspark.sql import DataFrame
from pyspark.sql.types import StructType, StringType

def camel_to_snake(name: str) -> str:
    """Converts a single string from camelCase to snake_case."""
    if name is None:
        return None
    return re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()


def flatten_df(df: DataFrame, cols_to_snake_case_values: set[str] | None = None) -> DataFrame:
    if cols_to_snake_case_values is None:
        cols_to_snake_case_values = set()

    @pandas_udf(StringType())
    def camel_to_snake_vectorized(series: pd.Series) -> pd.Series:
        return series.str.replace(r'(?<!^)(?=[A-Z])', '_', regex=True).str.lower()
    
    flat_cols = []

    def get_flat_cols(schema: StructType, prefix: str = ""):
        for field in schema.fields:
            full_col_name = f"{prefix}.{field.name}" if prefix else field.name

            if isinstance(field.dataType, StructType):
                get_flat_cols(field.dataType, prefix=f"{full_col_name}")
            else:
                alias_name = camel_to_snake(full_col_name.replace(".", "_"))

                if alias_name in cols_to_snake_case_values and isinstance(field.dataType, StringType):
                    flat_cols.append(
                        camel_to_snake_vectorized(col(full_col_name)).alias(alias_name)
                    )
                else:
                    flat_cols.append(
                        col(full_col_name).alias(alias_name)
                    )

    try:
        get_flat_cols(df.schema)
        
        if not flat_cols:
            logging.warning("flatten_df resulted in no columns. Returning an empty DataFrame.")
            return df.sparkSession.createDataFrame([], schema=StructType([]))
            
        return df.select(flat_cols)

    except Exception as e:
        logging.error("An error occurred during DataFrame flattening.", exc_info=True)
        raise

In [5]:
from pyspark.sql.functions import explode

departure_exploded_df = (
    bronze_flights.select(
        explode("departures").alias("data"),
        "airport",
        "ingestion_hour",
    )
)

arrival_exploded_df = (
    bronze_flights.select(
        explode("arrivals").alias("data"),
        "airport",
        "ingestion_hour",
    )
)

In [6]:
from pyspark.sql.functions import coalesce, concat_ws, lower, trim, lit, xxhash64, date_trunc

flattened_dep_df = flatten_df(
    departure_exploded_df.select(
        "data.*",
        "airport",
        "ingestion_hour"
    )
)
flattened_dep_df = transform_timestamps(flattened_dep_df)

enriched_dep_df = (
    flattened_dep_df
    .withColumn("departure_airport_sk", xxhash64(col("airport")))
    .withColumn(
        "arrival_airport_sk",
        xxhash64(
            coalesce(
                col("arrival_airport_iata"),
                col("arrival_airport_icao"),
                lower(trim(col("arrival_airport_name")))
            )
        )
    )
    .withColumn(
        "best_scheduled_time_utc",
        coalesce(col("departure_scheduled_time_utc"), col("arrival_scheduled_time_utc"))
    )
    .withColumn(
        "flight_composite_pk",
        concat_ws(
            "|",
            col("best_scheduled_time_utc"),
            col("number"),
            col("departure_airport_sk"),
            col("arrival_airport_sk")
        )
    )
    .withColumn(
        "flight_sk",
        xxhash64(col("flight_composite_pk"))
    )
)

flattened_arr_df = flatten_df(
    arrival_exploded_df.select(
        "data.*",
        "airport",
        "ingestion_hour"
    )
)
flattened_arr_df = transform_timestamps(flattened_arr_df)

enriched_arr_df = (
    flattened_arr_df
    .withColumn(
        "departure_airport_sk",
        xxhash64(
            coalesce(
                col("departure_airport_iata"),
                col("departure_airport_icao"),
                lower(trim(col("departure_airport_name")))
            )
        )

    )
    .withColumn("arrival_airport_sk", xxhash64(col("airport")))
    .withColumn(
        "best_scheduled_time_utc",
        coalesce(col("departure_scheduled_time_utc"), col("arrival_scheduled_time_utc"))
    )
    .withColumn(
        "flight_composite_pk",
        concat_ws(
            "|",
            col("best_scheduled_time_utc"),
            col("number"),
            col("departure_airport_sk"),
            col("arrival_airport_sk")
        )
    )
    .withColumn(
        "flight_sk",
        xxhash64(col("flight_composite_pk"))
    )
)

In [10]:
flattened_dep_df.show(5)

+----------------------+----------------------+----------------------------+----------------------+----------------------------+-----------------------------------+------------------------------+------------------------------+------------------------------+---------------------------+----------------------------+------------------------------+--------------------------+----------------------------+----------------------------+------------------------------+-------------------------+---------------------------+------------------+-----------------------+--------------+----------------------+----------------+-----------------+--------------------+--------------------+--------------------------+--------------------+--------------------------+---------------------------------+----------------------------+----------------------------+----------------------------+-------------------------+--------------------------+----------------------------+------------------------+------------------------

In [12]:
col_names = []
for c in flattened_arr_df.columns:
    col_names.append(c)

import pyspark.sql.functions as F

flattened_arr_df.select([
    F.count(F.col(c)).alias(c) for c in col_names
]).show(5, truncate=False)

+----------------------+----------------------+----------------------------+----------------------+----------------------------+-----------------------------------+------------------------------+------------------------------+------------------------------+---------------------------+----------------------------+------------------------------+--------------------------+----------------------------+----------------------------+------------------------------+-------------------------+---------------------------+------------------+-----------------------+--------------+----------------------+----------------+-----------------+--------------------+--------------------+--------------------------+--------------------+--------------------------+---------------------------------+----------------------------+----------------------------+----------------------------+-------------------------+--------------------------+----------------------------+------------------------+------------------------

In [32]:
import importlib
import utils.expression_utils  # make sure the module is imported
importlib.reload(utils.expression_utils)

from utils.expression_utils import get_select_expressions


In [7]:
from utils.expression_utils import get_select_expressions
from pyspark.sql.functions import when, length

union_flights = enriched_dep_df.union(enriched_arr_df).dropDuplicates(["flight_sk"])

# Airline iata codes are pretty inconsistent
enriched_union_flights = (
    union_flights
    .withColumn(
        "airline_sk",
        xxhash64(
            coalesce(
                col("airline_iata"),
                col("airline_icao"),
                when(
                    (length(col("airline_name")).between(3, 4)) & 
                    (col("airline_name").rlike("^[A-Z]+$")),
                    col("airline_name")             # treat as code
                ).otherwise(
                    lower(trim(col("airline_name")))
                )
            )
        )
    )

    .withColumn(
        "aircraft_sk",
        xxhash64(
            when(
                col("aircraft_reg").isNull() & 
                col("aircraft_mode_s").isNull() & 
                col("aircraft_model").isNull(),
                lit("UNKNOWN")
            ).otherwise(
                coalesce(
                    col("aircraft_reg"),
                    col("aircraft_mode_s"),
                    col("aircraft_model"),
                )
            )
        )
    ).withColumn(
        "aircraft_sk_type",
        when(col("aircraft_reg").isNotNull(), lit("reg"))
        .when(col("aircraft_mode_s").isNotNull(), lit("mode_s"))
        .when(col("aircraft_model").isNotNull(), lit("model"))
        .otherwise(lit("unknown"))
    )

    .withColumn(
        "dep_local_timezone",
        regexp_extract(col("departure_scheduled_time_local"), r"([+-]\d{2}:\d{2}|Z)$", 1)
    ).withColumn(
        "arr_local_timezone",
        regexp_extract(col("arrival_scheduled_time_local"), r"([+-]\d{2}:\d{2}|Z)$", 1)
    )
)

enriched_union_flights.printSchema()

root
 |-- departure_airport_icao: string (nullable = true)
 |-- departure_airport_iata: string (nullable = true)
 |-- departure_airport_local_code: string (nullable = true)
 |-- departure_airport_name: string (nullable = true)
 |-- departure_airport_short_name: string (nullable = true)
 |-- departure_airport_municipality_name: string (nullable = true)
 |-- departure_airport_location_lat: double (nullable = true)
 |-- departure_airport_location_lon: double (nullable = true)
 |-- departure_airport_country_code: string (nullable = true)
 |-- departure_airport_time_zone: string (nullable = true)
 |-- departure_scheduled_time_utc: timestamp (nullable = true)
 |-- departure_scheduled_time_local: string (nullable = true)
 |-- departure_revised_time_utc: timestamp (nullable = true)
 |-- departure_revised_time_local: string (nullable = true)
 |-- departure_predicted_time_utc: timestamp (nullable = true)
 |-- departure_predicted_time_local: string (nullable = true)
 |-- departure_runway_time_utc

In [31]:
enriched_union_flights.show(5, truncate=False)

+----------------------+----------------------+----------------------------+----------------------+----------------------------+-----------------------------------+------------------------------+------------------------------+------------------------------+---------------------------+----------------------------+------------------------------+--------------------------+----------------------------+----------------------------+------------------------------+-------------------------+---------------------------+------------------+-----------------------+--------------+----------------------+----------------+-----------------+--------------------+--------------------+--------------------------+--------------------+--------------------------+---------------------------------+----------------------------+----------------------------+----------------------------+-------------------------+--------------------------+----------------------------+------------------------+------------------------

In [8]:
print(fct_flights.count())
a = fct_flights.dropDuplicates(['flight_pk'])
print(a.count())

25/10/06 08:50:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


101104
101104


In [16]:
fct_flights.printSchema()

root
 |-- flight_pk: long (nullable = false)
 |-- flight_composite_pk: string (nullable = false)
 |-- departure_airport_pk: string (nullable = true)
 |-- arrival_airport_pk: string (nullable = true)
 |-- airline_pk: string (nullable = true)
 |-- aircraft_pk: string (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- flight_callsign: string (nullable = true)
 |-- flight_status: string (nullable = true)
 |-- codeshare_status: string (nullable = true)
 |-- is_cargo: boolean (nullable = true)
 |-- dep_scheduled_at_utc: timestamp (nullable = true)
 |-- dep_revised_at_utc: timestamp (nullable = true)
 |-- dep_predicted_at_utc: timestamp (nullable = true)
 |-- dep_runway_at_utc: timestamp (nullable = true)
 |-- departure_scheduled_time_local_ntz: timestamp_ntz (nullable = true)
 |-- departure_revised_time_local_ntz: timestamp_ntz (nullable = true)
 |-- departure_predicted_time_local_ntz: timestamp_ntz (nullable = true)
 |-- departure_runway_time_local_ntz: timestamp_ntz (nulla

In [20]:
fct_flights.show(truncate=False)

+---------------------------------------------------------------------+--------------------+--------------------+--------------------+--------------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+-------------+------------+--------+----------------+----------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_composite_pk                                                  |departure_airport_sk|arrival_airport_sk  |airline_sk          |aircraft_sk         |flight_number|flight_callsign|flight_status|codeshare_status|is_cargo|d

In [9]:
enriched_union_flights.show(5)

+----------------------+----------------------+----------------------------+----------------------+----------------------------+-----------------------------------+------------------------------+------------------------------+------------------------------+---------------------------+----------------------------+------------------------------+--------------------------+----------------------------+----------------------------+------------------------------+-------------------------+---------------------------+------------------+-----------------------+--------------+----------------------+----------------+-----------------+--------------------+--------------------+--------------------------+--------------------+--------------------------+---------------------------------+----------------------------+----------------------------+----------------------------+-------------------------+--------------------------+----------------------------+------------------------+------------------------

In [29]:
col_names = []
for c in fct_flights.columns:
    col_names.append(c)

from pyspark.sql import functions as sf

fct_flights.select([
    sf.count(sf.col(c)).alias(c) for c in col_names
]).show()

+---------+-------------------+--------------------+------------------+----------+-----------+-------------+---------------+-------------+----------------+--------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+------------+----------------+--------+----------------+----------+-----------+------------+----------------+--------+----------------+----------+-----------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_pk|flight_composite_pk|departure_airport_pk|arrival_airport_pk|airline_pk|aircraft_pk|flight_number|flight_callsign|flight_status|codeshare_status|is_cargo|dep

In [36]:
col_names = []
for c in fct_flights.columns:
    col_names.append(c)

from pyspark.sql import functions as sf

fct_flights.select([
    sf.count(sf.col(c)).alias(c) for c in col_names
]).show()

+---------+-------------------+--------------------+------------------+----------+-----------+-------------+---------------+-------------+----------------+--------+--------------------+------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+------------+----------------+--------+----------------+----------+-----------+------------+----------------+--------+----------------+----------+-----------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_pk|flight_composite_pk|departure_airport_pk|arrival_airport_pk|airline_pk|aircraft_pk|flight_number|flight_callsign|flight_status|codeshare_status|is_cargo|dep_scheduled_at_utc|dep

In [27]:
col_names = []
for c in fct_flights.columns:
    col_names.append(c)

from pyspark.sql import functions as sf

fct_flights.select([
    sf.count(sf.col(c)).alias(c) for c in col_names
]).show()

+---------+-------------------+--------------------+------------------+----------+-----------+-------------+---------------+-------------+----------------+--------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+------------+----------------+--------+----------------+----------+-----------+------------+----------------+--------+----------------+----------+-----------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_pk|flight_composite_pk|departure_airport_pk|arrival_airport_pk|airline_pk|aircraft_pk|flight_number|flight_callsign|flight_status|codeshare_status|is_cargo|dep

In [17]:
# fct_flights.filter(col("dep_runway_at_utc").isNull() & (col("ingestion_hour") != "2025-10-02-12")).show(10)
fct_flights.filter(col("departure_scheduled_time_utc").isNull() & col("arrival_scheduled_time_utc").isNull()).show(10)

+---------+-------------------+--------------------+------------------+----------+------------+-------------+---------------+-------------+----------------+--------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+------------+--------+------------+----------------+-----------+-----------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_pk|flight_composite_pk|departure_airport_pk|arrival_airport_pk|airline_pk|aircraft_reg|flight_number|flight_callsign|flight_status|codeshare_status|is_cargo|dep_scheduled_at_utc|dep_revised_at_utc|dep_predicted_at_utc|dep_runway_at_utc|dep_

In [9]:
from pyspark.sql.functions import col, count, desc, asc

freq = (
    fct_flights.groupBy("flight_pk")
    .agg(count("*").alias("count"))
    .orderBy(desc("count"))
)

freq.show()

25/10/08 08:30:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----+
|           flight_pk|count|
+--------------------+-----+
|  984243367990722308|    1|
|-6641328797403377180|    1|
|-1872682190123097138|    1|
| 8335349881670655812|    1|
|-1929753405681982598|    1|
| 7944061740591960284|    1|
|-3056829225050653113|    1|
| 8842538117914831012|    1|
| 5447812882236428322|    1|
|-3904159345978257643|    1|
| 1734181022247872282|    1|
|-3581317485510168370|    1|
|-3298283055477450332|    1|
|-4180815870992356600|    1|
|-8943426624468755770|    1|
| 8450030353303618229|    1|
|-6687152434929302686|    1|
|-1093361121475283209|    1|
|-8218639077838886567|    1|
|-6377462062338467371|    1|
+--------------------+-----+
only showing top 20 rows



In [35]:
top_flight_number = freq.collect()[0]["flight_pk"]

all_records_top_flight = fct_flights.filter(col("flight_pk") == -5535522808985056397)

all_records_top_flight.show(20, truncate=False)

+--------------------+-------------------------------+--------------------+------------------+----------+------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+--------------------+-------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+------------+--------+------------+----------------+-------------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_pk           |flight_composite_pk            |departure_airport_pk|arrival_airport_pk|airline_pk|aircraft_reg|flight_number|flight_callsign|flight_status|codeshare_status|is_cargo|dep_scheduled_at_utc|dep_revise

In [10]:
fct_flights.count()

120593

In [50]:
from pyspark.sql.functions import col

top30_codes = [
    "LHR","IST","CDG","AMS","MAD","FRA","BCN","FCO","SVO","LGW",
    "MUC","LIS","DUB","PMI","ORY","ATH","VIE","ZRH","MAN","CPH",
    "STN","MXP","OSL","BER","AGP","BRU","ARN","WAW","LED","DUS"
]


check = (
    fct_flights
    .select("flight_pk", "departure_airport_pk", "arrival_airport_pk")
    .distinct()
    .withColumn("dep_in_top30", col("departure_airport_pk").isin(top30_codes))
    .withColumn("arr_in_top30", col("arrival_airport_pk").isin(top30_codes))
)

check.filter(col("dep_in_top30") & col("arr_in_top30")).count()


21223

In [15]:
col_names = []
for c in flattened_dep_df.columns:
    col_names.append(c)

from pyspark.sql import functions as sf

flattened_dep_df.select([
    sf.count(sf.col(c)).alias(c) for c in col_names
]).show()

flattened_dep_df.show(1)

+----------------------+----------------------+----------------------------+----------------------+----------------------------+-----------------------------------+------------------------------+------------------------------+------------------------------+---------------------------+----------------------------+------------------------------+--------------------------+----------------------------+----------------------------+------------------------------+-------------------------+---------------------------+------------------+-----------------------+--------------+----------------------+----------------+-----------------+--------------------+--------------------+--------------------------+--------------------+--------------------------+---------------------------------+----------------------------+----------------------------+----------------------------+-------------------------+--------------------------+----------------------------+------------------------+------------------------

In [54]:
flattened_dep_df.show(5)

+----------------------+----------------------+----------------------------+----------------------+----------------------------+-----------------------------------+------------------------------+------------------------------+------------------------------+---------------------------+----------------------------+------------------------------+--------------------------+----------------------------+----------------------------+------------------------------+-------------------------+---------------------------+------------------+-----------------------+--------------+----------------------+----------------+-----------------+--------------------+--------------------+--------------------------+--------------------+--------------------------+---------------------------------+----------------------------+----------------------------+----------------------------+-------------------------+--------------------------+----------------------------+------------------------+------------------------

In [8]:
# bronze_airports.show(5)

flattened_airports_df = flatten_df(bronze_airports)

detailed_airports_select_exprs = get_select_expressions("silver", "dim_airports_detailed")

dim_airports_detailed = flattened_airports_df.select(*detailed_airports_select_exprs)
dim_airports_detailed.show()


25/10/10 13:35:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+----+--------------------+--------------------+-----------------+---------+---------+--------------+------------+--------------+--------------+--------------+-----------------+
|iata|icao|          short_name|           full_name|municipality_name| latitude|longitude|elevation_feet|country_code|  country_name|continent_code|continent_name|        time_zone|
+----+----+--------------------+--------------------+-----------------+---------+---------+--------------+------------+--------------+--------------+--------------+-----------------+
| AMS|EHAM|            Schiphol|  Amsterdam Schiphol|        Amsterdam|  52.3086| 4.763889|         -11.0|          NL|   Netherlands|            EU|        Europe| Europe/Amsterdam|
| MAD|LEMD|Adolfo Suárez –Ba...|Madrid Adolfo Suá...|           Madrid|  40.4936| -3.56676|        1998.0|          ES|         Spain|            EU|        Europe|    Europe/Madrid|
| CDG|LFPG|   Charles de Gaulle|Paris Charles de ...|            Paris|  49.0128| 2.5

In [13]:
test_cols = []
for c in test.columns:
    test_cols.append(c)

import pyspark.sql.functions as sf

test.select([
    sf.count(sf.col(c)).alias(c) for c in test_cols
]).show()

+----+----+----------+----------+---------+-----------------+------------+------------+---------------+------------+--------------+------------+--------------+------------+------------+--------------+--------------+---------+-------------+--------------+------------+-------------+-----------------+----------------+-------+---------------------+-----------------------+-------------------------+-------+---------------------------+
|icao|iata|local_code|short_name|full_name|municipality_name|location_lat|location_lon|elevation_meter|elevation_km|elevation_mile|elevation_nm|elevation_feet|country_code|country_name|continent_code|continent_name|time_zone|urls_web_site|urls_wikipedia|urls_twitter|urls_live_atc|urls_flight_radar|urls_google_maps|runways|current_time_time_utc|current_time_time_local|current_time_time_zone_id|airport|current_time_time_local_ntz|
+----+----+----------+----------+---------+-----------------+------------+------------+---------------+------------+--------------+---

In [25]:
enriched_union_flights.printSchema()

root
 |-- departure_airport_icao: string (nullable = true)
 |-- departure_airport_iata: string (nullable = true)
 |-- departure_airport_local_code: string (nullable = true)
 |-- departure_airport_name: string (nullable = true)
 |-- departure_airport_short_name: string (nullable = true)
 |-- departure_airport_municipality_name: string (nullable = true)
 |-- departure_airport_location_lat: double (nullable = true)
 |-- departure_airport_location_lon: double (nullable = true)
 |-- departure_airport_country_code: string (nullable = true)
 |-- departure_airport_time_zone: string (nullable = true)
 |-- departure_scheduled_time_utc: timestamp (nullable = true)
 |-- departure_scheduled_time_local: string (nullable = true)
 |-- departure_revised_time_utc: timestamp (nullable = true)
 |-- departure_revised_time_local: string (nullable = true)
 |-- departure_predicted_time_utc: timestamp (nullable = true)
 |-- departure_predicted_time_local: string (nullable = true)
 |-- departure_runway_time_utc

In [9]:
# Departures usually don't include arrival airport details
arr_airports = (
    enriched_dep_df.select(
        col("arrival_airport_sk").alias("airport_sk"),
        col("arrival_airport_iata").alias("airport_iata"),
        col("arrival_airport_icao").alias("airport_icao"),
        col("arrival_airport_name").alias("airport_name"),
        col("arrival_airport_time_zone").alias("airport_time_zone"),
    )
)

# Same goes for arrivals missing departure airport details
dep_airports = (
    enriched_arr_df.select(
        col("departure_airport_sk").alias("airport_sk"),
        col("departure_airport_iata").alias("airport_iata"),
        col("departure_airport_icao").alias("airport_icao"),
        col("departure_airport_name").alias("airport_name"),
        col("departure_airport_time_zone").alias("airport_time_zone"),
    )
)

dim_airports_basic = dep_airports.union(arr_airports).dropDuplicates(["airport_sk"])
print(dim_airports_basic.count())

dim_airports_basic.filter(col("airport_name") == "London").show(20, truncate=False)
# dim_airports_basic.filter(col("airport_iata").isNull()).show(200, truncate=False)
dim_airports_basic.filter(col("airport_iata").isNotNull()).count()

# dim_airports_basic.write.mode("overwrite").csv("/home/dottier/flights_etl/data/airports.csv", header=True)

1142


+--------------------+------------+------------+------------+-----------------+
|airport_sk          |airport_iata|airport_icao|airport_name|airport_time_zone|
+--------------------+------------+------------+------------+-----------------+
|-7580325845725858894|LHR         |EGLL        |London      |Europe/London    |
|-6160843054774251004|LGW         |EGKK        |London      |Europe/London    |
|-5091017362067582158|NULL        |NULL        |London      |NULL             |
|-1166757016042454982|LTN         |EGGW        |London      |Europe/London    |
|3936163360546436249 |LCY         |EGLC        |London      |Europe/London    |
|4262365550847078087 |BQH         |EGKB        |London      |Europe/London    |
|6915843395098580572 |STN         |EGSS        |London      |Europe/London    |
|9176707735994611623 |NHT         |EGWU        |London      |Europe/London    |
+--------------------+------------+------------+------------+-----------------+



880

In [10]:
from pyspark.sql.functions import col, coalesce, xxhash64

# Perform a left join to combine the two datasets on the IATA code.
enriched_df = dim_airports_basic.alias("basic").join(
    dim_airports_detailed.alias("detailed"),
    on=col("basic.airport_iata") == col("detailed.iata"), # Assuming 'iata' is the key in detailed
    how="left"
)

enriched_df.printSchema()

# 2. Define the final, authoritative dimension table using the "Coalesce and Select" pattern.
#    This is the most important step.
dim_airports_final = enriched_df.select(
    # --- Keys and Identifiers ---
    # The PK comes from the basic table, ensuring all airports have one.
    col("basic.airport_sk"),
    col("basic.airport_iata"),
    coalesce(col("detailed.icao"), col("basic.airport_icao")).alias("airport_icao"),

    # --- Naming and Location ---
    # Prefer the detailed, full name, but fall back to the basic name.
    coalesce(col("detailed.full_name"), col("basic.airport_name")).alias("airport_name"),
    col("detailed.municipality_name"), # This only exists in the detailed source
    col("detailed.country_name"),
    col("detailed.continent_name"),
    
    # --- Technical Specs ---
    # The detailed source is the only one with this info. Nulls are acceptable.
    col("detailed.latitude"),
    col("detailed.longitude"),
    col("detailed.elevation_feet"),
    
    # --- Timezone ---
    # Prefer the detailed timezone, but fall back to the basic one.
    coalesce(col("detailed.time_zone"), col("basic.airport_time_zone")).alias("airport_time_zone")
)

# Now, dim_airports_final is a single, clean, authoritative table.
dim_airports_final.show()

root
 |-- airport_sk: long (nullable = false)
 |-- airport_iata: string (nullable = true)
 |-- airport_icao: string (nullable = true)
 |-- airport_name: string (nullable = true)
 |-- airport_time_zone: string (nullable = true)
 |-- iata: string (nullable = true)
 |-- icao: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- municipality_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elevation_feet: double (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- continent_code: string (nullable = true)
 |-- continent_name: string (nullable = true)
 |-- time_zone: string (nullable = true)

+--------------------+------------+------------+--------------------+-----------------+------------+--------------+--------+---------+--------------+--------------------+
|          airport_sk|airport_iata|airport_icao| 

dim_runways

In [39]:
r = flattened_airports_df.select("runways")
r.printSchema()

root
 |-- runways: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- trueHdg: double (nullable = true)
 |    |    |-- length: struct (nullable = true)
 |    |    |    |-- meter: double (nullable = true)
 |    |    |    |-- km: double (nullable = true)
 |    |    |    |-- mile: double (nullable = true)
 |    |    |    |-- nm: double (nullable = true)
 |    |    |    |-- feet: double (nullable = true)
 |    |    |-- width: struct (nullable = true)
 |    |    |    |-- meter: double (nullable = true)
 |    |    |    |-- km: double (nullable = true)
 |    |    |    |-- mile: double (nullable = true)
 |    |    |    |-- nm: double (nullable = true)
 |    |    |    |-- feet: double (nullable = true)
 |    |    |-- isClosed: boolean (nullable = true)
 |    |    |-- location: struct (nullable = true)
 |    |    |    |-- lat: double (nullable = true)
 |    |    |    |-- lon: double (nullable = true)
 |    |    |--

In [44]:
flattened_runways_df.printSchema()

root
 |-- airport_pk: string (nullable = true)
 |-- name: string (nullable = true)
 |-- true_hdg: double (nullable = true)
 |-- length_meter: double (nullable = true)
 |-- length_km: double (nullable = true)
 |-- length_mile: double (nullable = true)
 |-- length_nm: double (nullable = true)
 |-- length_feet: double (nullable = true)
 |-- width_meter: double (nullable = true)
 |-- width_km: double (nullable = true)
 |-- width_mile: double (nullable = true)
 |-- width_nm: double (nullable = true)
 |-- width_feet: double (nullable = true)
 |-- is_closed: boolean (nullable = true)
 |-- location_lat: double (nullable = true)
 |-- location_lon: double (nullable = true)
 |-- surface: string (nullable = true)
 |-- displaced_threshold_meter: double (nullable = true)
 |-- displaced_threshold_km: double (nullable = true)
 |-- displaced_threshold_mile: double (nullable = true)
 |-- displaced_threshold_nm: double (nullable = true)
 |-- displaced_threshold_feet: double (nullable = true)
 |-- has_lig

In [11]:
from pyspark.sql.functions import explode, col, concat_ws, round as spark_round

exploded_runways = (
    flattened_airports_df.select(
        col("iata").alias("airport_pk"),
        explode("runways").alias("data")
    )
)

flattened_runways_df = flatten_df(
    exploded_runways.select(
        "airport_pk",
        "data.*"
    )
)

flattened_runways_df = flattened_runways_df.withColumn(
    "airport_sk",
    xxhash64(col("airport_pk"))    
)

unknown_runway = [
    {"airport_sk": -1, "runway_name": "UNKNOWN", "true_heading": 0.0, "surface": "Unknown",
     "has_lighting": False, "is_closed": False, "length_feet": 0.0, "width_feet": 0.0, "displaced_threshold_feet": 0.0,
     "latitude": 0.0, "longitude": 0.0}
]
unknown_runway_df = spark.createDataFrame(unknown_runway)

r_select_exprs = get_select_expressions("silver", "dim_runways")
dim_runways = flattened_runways_df.select(*r_select_exprs)
dim_runways = dim_runways.unionByName(unknown_runway_df)

dim_runways.show()
dim_runways.count()

+-------------------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+---------+----------+
|         airport_sk|runway_name|true_heading|surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet| latitude| longitude|
+-------------------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+---------+----------+
|3783112611505365340|         04|        41.1|Asphalt|        true|    false|     6634.0|     148.0|                     0.0|52.300358|  4.783485|
|3783112611505365340|         06|        57.9|Asphalt|        true|    false|    11283.0|     148.0|                  820.21| 52.28792| 4.7341533|
|3783112611505365340|         09|        86.8|Asphalt|        true|    false|    11306.0|     148.0|                  291.99|52.316628| 4.7463284|
|3783112611505365340|        18C|       183.2|Asphalt|        true|    false|    10833.0|     148.0|                  

163

In [ ]:
dim_airports.count()
dim_airports.filter(col("iata") == "LHR").show(5)

+----+----+----------+----------+---------------+-----------------+------------+------------+---------------+------------+--------------+------------+--------------+------------+--------------+--------------+--------------+-------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+---------------------+-----------------------+-------------------------+-------+----------+------------+------------+------------+-----------------+
|icao|iata|local_code|short_name|      full_name|municipality_name|location_lat|location_lon|elevation_meter|elevation_km|elevation_mile|elevation_nm|elevation_feet|country_code|  country_name|continent_code|continent_name|    time_zone|       urls_web_site|      urls_wikipedia|        urls_twitter|urls_live_atc|   urls_flight_radar|    urls_google_maps|             runways|current_time_time_utc|current_time_time_local|current_time_time_zone_id|airport|airport_pk|airpor

In [12]:
dim_airlines = enriched_union_flights.select(
    col("airline_sk").alias("airline_sk"),
    col("airline_iata").alias("airline_iata"),
    col("airline_icao").alias("airline_icao"),
    col("airline_name").alias("airline_name"),
).dropDuplicates(["airline_sk"])

dim_airlines.show(5)
# dim_airlines.orderBy(length("airline_sk").desc()).show(20, truncate=False)

+--------------------+------------+------------+--------------+
|          airline_sk|airline_iata|airline_icao|  airline_name|
+--------------------+------------+------------+--------------+
|-9181349260121889611|          SI|         BCI|  Blue Islands|
|-9157863550347410489|          J1|        NULL|       One Jet|
|-9155545459179649926|          EK|         UAE|      Emirates|
|-9141596393363223783|          3F|         FIE|FlyOne Armenia|
|-9073928079885810969|        NULL|        NULL|            W4|
+--------------------+------------+------------+--------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import countDistinct, col
from pyspark.sql.functions import when, length

union_flights = enriched_dep_df.union(enriched_arr_df)

(
    dim_airports
    .groupBy("airport_name")
    .agg(
        countDistinct("airport_iata").alias("distinct_iata"),
        countDistinct("airport_icao").alias("distinct_icao"),
        countDistinct("airport_time_zone").alias("distinct_tz"),
    )
    # .filter(
    #     (col("distinct_iata") >= 1) | (col("distinct_icao") >= 1) | (col("distinct_tz") >= 1)
    # )
    .orderBy(col("distinct_iata").asc())
    .show(truncate=False)
)


+--------------------+-------------+-------------+-----------+
|airport_name        |distinct_iata|distinct_icao|distinct_tz|
+--------------------+-------------+-------------+-----------+
|Paros               |0            |0            |0          |
|Sint-Marten         |0            |0            |0          |
|Kos                 |0            |0            |0          |
|Frankfurt Hahn      |0            |0            |0          |
|Kluzh               |0            |0            |0          |
|Chisinau            |0            |0            |0          |
|Venice, Marco Polo  |0            |0            |0          |
|Speyer              |0            |0            |0          |
|Villafranca Airport |0            |0            |0          |
|Jersey              |0            |0            |0          |
|Paris, Ch. de Gaulle|0            |0            |0          |
|Birrfeld            |0            |0            |0          |
|Mitilini            |0            |0            |0    

In [26]:
dim_aircraft = enriched_union_flights.select(
    "aircraft_sk",
    "aircraft_reg",
    "aircraft_mode_s",
    "aircraft_model",
    "aircraft_sk_type"
).dropDuplicates(["aircraft_sk"])

dim_aircraft.filter(col("aircraft_sk_type") == "model").show(100)
dim_aircraft.count()

+--------------------+------------+---------------+--------------------+----------------+
|         aircraft_sk|aircraft_reg|aircraft_mode_s|      aircraft_model|aircraft_sk_type|
+--------------------+------------+---------------+--------------------+----------------+
|-9211695441214389923|        NULL|           NULL|Airbus A-320-200 ...|           model|
|-9077051277013598409|        NULL|           NULL|                 W38|           model|
|-8731780478934453966|        NULL|           NULL|Boeing 737-800 (w...|           model|
|-8729229220927507874|        NULL|           NULL|Bombardier Dash 8...|           model|
|-8384223938789297699|        NULL|           NULL|                E295|           model|
|-8244485509375181072|        NULL|           NULL|                 W37|           model|
|-8129694338162631332|        NULL|           NULL|    Boeing 777-300ER|           model|
|-8125499267023080516|        NULL|           NULL|      Boeing 757-200|           model|
|-79156666

7893

In [38]:
from pyspark.sql.functions import col

null_aircraft = enriched_union_flights.filter(
    col("aircraft_reg").isNull() &
    col("aircraft_mode_s").isNull() &
    col("aircraft_model").isNotNull()
)

null_aircraft.show(truncate=False)
null_aircraft.count()


+----------------------+----------------------+----------------------------+----------------------+----------------------------+-----------------------------------+------------------------------+------------------------------+------------------------------+---------------------------+----------------------------+------------------------------+--------------------------+----------------------------+----------------------------+------------------------------+-------------------------+---------------------------+------------------+-----------------------+--------------+----------------------+----------------+-----------------+--------------------+--------------------+--------------------------+--------------------+--------------------------+---------------------------------+----------------------------+----------------------------+----------------------------+-------------------------+--------------------------+----------------------------+------------------------+------------------------

47106

In [ ]:
from pyspark.sql.functions import col

# df is your original DataFrame
df1 = fct_flights.alias("df1")
df2 = fct_flights.alias("df2")

df = df1.join(
    df2,
    on=(
        df1["departure_airport_iata"] == df2["arrival_airport_iata"] &
        df1["arrival_airport_iata"] == df2["departure_airport_iata"] &
        df1[]
    )
)

In [11]:
fct_flights.filter(col("airline_icao").isNull() & col("airline_iata").isNull()).show(5)

+--------------------+--------------------+----------------------+--------------------+------------+------------+------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+--------------------+-------------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+------------+--------+------------+----------------+-------------+-----------+-----------------------------+---------+---------+-----------+----------------+--------------+--------------+
|           flight_pk| flight_composite_pk|departure_airport_iata|arrival_airport_iata|airline_iata|airline_icao|aircraft_reg|flight_number|flight_callsign|flight_status|codeshare_status|is_cargo|dep_scheduled_a

In [20]:
import pyspark.sql.functions as sf

fct_flights.filter(sf.length(col("airline_pk")) > 4).show(5, truncate=False)

+--------------------+------------------------------------------------------------+----------------------+---------------------------+---------------------------+------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+--------------------+-------------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+--------------------+------------------+--------------------+-----------------+--------------------------+------------------------+--------------------------+-----------------------+------------------+------------+--------+------------+----------------+-------------+-----------+-----------------------------+---------+---------+-----------+----------------+--------------+--------------+
|flight_pk           |flight_composite_pk                                         |departure_airport_iata|arrival_airport_iata       |airline_pk                 |

In [2]:
from process.utils import write_delta_table

spark.sql("DROP DATABASE IF EXISTS silver CASCADE")
spark.sql("CREATE DATABASE IF NOT EXISTS silver")
spark.sql("SHOW TABLES IN silver").show()

# write_delta_table(spark, fct_flights, "silver", "fct_flights", partition_cols=["ingestion_hour"])
write_delta_table(spark, dim_airports_final, "silver", "dim_airports", write_mode="merge", primary_keys=["airport_pk"])
# write_delta_table(dim_runways, "silver", "dim_runways", write_mode="merge", primary_keys=["airport_pk", "surface", "length_ft", "width_ft"])
write_delta_table(spark, dim_airlines, "silver", "dim_airlines", write_mode="merge", primary_keys=["airline_pk"])
write_delta_table(spark, dim_aircraft, "silver", "dim_aircraft", write_mode="merge", primary_keys=["aircraft_pk"])



ModuleNotFoundError: No module named 'delta'

In [14]:
from delta.tables import DeltaTable
import logging
from datetime import datetime, timezone
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import sha2, concat_ws, current_timestamp
import functools

def write_delta_table_with_scd(
    spark: SparkSession,
    df: DataFrame,
    db_name: str,
    table_name: str,
    business_keys: list[str],
    surrogate_key: str,
):
    full_table_name = f"{db_name}.{table_name}"
    table_path = f"/home/dottier/flights_etl/{db_name}/{table_name}"

    if not business_keys or surrogate_key is None:
        raise ValueError("Business keys and surrogate key must be provided for SCD Type 2 merge.")

    logging.info(f"--- Preparing to write to Delta table: {full_table_name} ---")

    now = current_timestamp()

    try:
        if not spark.catalog.tableExists(full_table_name):
            logging.info(f"Table {full_table_name} does not exist. Creating new Delta table.")

            df = df.withColumn(
                "effective_start_date", now
            ).withColumn(
                "effective_end_date", lit(None).cast("timestamp")
            )

            df = df.select(
                xxhash64(concat_ws("|", *business_keys, now)).alias(surrogate_key),
                *df.columns,
            )
            
            df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(table_path)

            logging.info(f"Registering Delta table {full_table_name} at location {table_path}.")
            spark.sql(f"CREATE TABLE {full_table_name} USING DELTA LOCATION '{table_path}'")
            logging.info(f"Table {full_table_name} created successfully.")

            return
        
        logging.info(f"Table {full_table_name} exists. Performing SCD Type 2 merge.")
        target_table = DeltaTable.forName(spark, full_table_name)

        non_key_columns = [col for col in df.columns if col not in business_keys]
        source_df_with_hash = df.withColumn(
            "row_hash",
            sha2(concat_ws("|", *non_key_columns), 256)
        )

        source_df_with_hash.printSchema()

        target_df_with_hash = (
            target_table.toDF()
            .filter(col("effective_end_date").isNull())
            .withColumn(
                "row_hash",
                sha2(concat_ws("|", *non_key_columns), 256)
            )
        )

        target_df_with_hash.printSchema()

        join_condition = [
            col(f"source.{key}") == col(f"target.{key}") for key in business_keys
        ]
        final_join_condition = functools.reduce(lambda x, y: x & y, join_condition) if join_condition else None

        staging_df = source_df_with_hash.alias("source").join(
            target_df_with_hash.alias("target"),
            on=final_join_condition,
            how="left"
        ).select(
            "source.*",
            col(f"target.{surrogate_key}").alias("existing_sk"),
            col("target.row_hash").alias("existing_hash")
        )

        expired_records = (
            staging_df
            .filter((col("existing_sk").isNotNull()) & (col("existing_hash") != col("row_hash")))
            .select(
                col("existing_sk").alias(surrogate_key),
                now.alias("effective_end_date")
            )
        )

        new_records = (
            staging_df
            .select(
                xxhash64(concat_ws("||", *business_keys, now)).alias(surrogate_key),
                *df.columns,
                now.alias("effective_start_date"),
                lit(None).cast("timestamp").alias("effective_end_date")
            )
        )

        final_change_set = new_records.unionByName(expired_records, allowMissingColumns=True)

        (
            target_table.alias("target")
            .merge(
                final_change_set.alias("source"),
                condition=f"target.{surrogate_key} = source.{surrogate_key}"
            )
            .whenNotMatchedInsertAll()
            .whenMatchedUpdate(
                set={"effective_end_date": "source.effective_end_date"}
            )
            .execute()
        )

        logging.info(f"SCD Type 2 merge into {full_table_name} completed successfully.")

    except Exception as e:
        logging.error(f"FATAL: Failed during WRITE phase for '{full_table_name}'. Error: {e}", exc_info=True)
        raise

In [17]:
dim_runways.show(5)

+-------------------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+---------+---------+
|         airport_sk|runway_name|true_heading|surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet| latitude|longitude|
+-------------------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+---------+---------+
|3783112611505365340|         04|        41.1|Asphalt|        true|    false|     6634.0|     148.0|                     0.0|52.300358| 4.783485|
|3783112611505365340|         06|        57.9|Asphalt|        true|    false|    11283.0|     148.0|                  820.21| 52.28792|4.7341533|
|3783112611505365340|         09|        86.8|Asphalt|        true|    false|    11306.0|     148.0|                  291.99|52.316628|4.7463284|
|3783112611505365340|        18C|       183.2|Asphalt|        true|    false|    10833.0|     148.0|                     0.0

In [24]:
flights_with_runway = fct_flights.filter(col("dep_runway").isNotNull() | col("arr_runway").isNotNull())
flights_with_runway.show()
flights_with_runway.count()

+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+-------------+------------+--------+----------------+----------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
| flight_composite_pk|departure_airport_sk|  arrival_airport_sk|          airline_sk|         aircraft_sk|flight_number|flight_callsign|flight_status|codeshare_status|is_cargo|dep_scheduled_at_utc| dep_revised_at_utc|  dep_runway_at_utc|dep_scheduled_at_local_ntz|dep_revised

4494

In [14]:
flights = flights_with_runway.alias("flights")
dep_runway = dim_runways.alias("dep")
arr_runway = dim_runways.alias("arr")

joined = flights.join(
    dep_runway,
    on=[col("flights.dep_runway") == col("dep.runway_name"), col("flights.departure_airport_sk") == col("dep.airport_sk")],
    how="left"
).join(
    arr_runway,
    on=[col("flights.arr_runway") == col("arr.runway_name"), col("flights.arrival_airport_sk") == col("arr.airport_sk")],
    how="left"  
)

NameError: name 'flights_with_runway' is not defined

In [47]:
# Alias the tables for clarity
flights = flights_with_runway.alias("flights")
arr_runway_dim = dim_runways.alias("arr")

# Perform a LEFT join
arrival_mismatches = flights.join(
    arr_runway_dim,
    on=[
        col("flights.arr_runway") == col("arr.runway_name"),
        col("flights.arrival_airport_sk") == col("arr.airport_sk")
    ],
    how="left"
).filter(
    # Condition 1: We only care about flights that HAD an arrival runway name
    col("flights.arr_runway").isNotNull() &

    # Condition 2: But the join FAILED (indicated by null on the dimension side)
    col("arr.true_heading").isNull()
)

print("Number of flights with arrival runway SK mismatch:")
arrival_mismatches.count()

Number of flights with arrival runway SK mismatch:


482

In [48]:
arrival_mismatches.show(5, truncate=False)

+--------------------------------------------------------------------+--------------------+--------------------+--------------------+--------------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+-------------+------------+--------+----------------+----------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+
|flight_composite_pk                                                 |departure_airport_sk|ar

In [50]:
dim_airports_final.filter(col("airport_sk") == 3936163360546436249 ).show(5)

+-------------------+------------+------------+------------+-----------------+------------+--------------+--------+---------+--------------+-----------------+
|         airport_sk|airport_iata|airport_icao|airport_name|municipality_name|country_name|continent_name|latitude|longitude|elevation_feet|airport_time_zone|
+-------------------+------------+------------+------------+-----------------+------------+--------------+--------+---------+--------------+-----------------+
|3936163360546436249|         LCY|        EGLC|      London|             NULL|        NULL|          NULL|    NULL|     NULL|          NULL|    Europe/London|
+-------------------+------------+------------+------------+-----------------+------------+--------------+--------+---------+--------------+-----------------+



In [46]:
# Alias the tables for clarity
flights = flights_with_runway.alias("flights")
dep_runway_dim = dim_runways.alias("dep")

# Perform a LEFT join
departure_mismatches = flights.join(
    dep_runway_dim,
    on=[
        col("flights.dep_runway") == col("dep.runway_name"),
        col("flights.departure_airport_sk") == col("dep.airport_sk")
    ],
    how="left"
).filter(
    # Condition 1: We only care about flights that HAD a departure runway name
    col("flights.dep_runway").isNotNull() 

    # Condition 2: But the join FAILED (indicated by null on the dimension side)
    # col("dep.true_heading").isNull()
)

print("Number of flights with departure runway SK mismatch:")
departure_mismatches.count()

Number of flights with departure runway SK mismatch:


3320

In [35]:
joined.show(5, truncate=False)

+--------------------------------------------------------------------+--------------------+--------------------+--------------------+--------------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+-------------+------------+--------+----------------+----------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+--------------------+-----------+------------+--------+------------+---------+-----------+----------+------------------------+---------+---------+--------------------+-----------+------------+-------+------------+---------+-----

In [ ]:
joined.filter(col("dep_runway").isNotNull() | col("arr_runway").isNotNull()).count()

510

In [89]:
dim_runways.count()

162

In [82]:
spark.sql("SHOW TABLES IN silver").show()
spark.sql("DROP TABLE IF EXISTS silver.dim_runways").show()
spark.sql("SHOW TABLES IN silver").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|   silver|dim_aircraft|      false|
|   silver|dim_airlines|      false|
|   silver|dim_airports|      false|
|   silver| dim_runways|      false|
|   silver| fct_flights|      false|
+---------+------------+-----------+

++
||
++
++

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|   silver|dim_aircraft|      false|
|   silver|dim_airlines|      false|
|   silver|dim_airports|      false|
|   silver| fct_flights|      false|
+---------+------------+-----------+



In [74]:
dim_runways.printSchema()

root
 |-- airport_sk: long (nullable = false)
 |-- runway_name: string (nullable = true)
 |-- true_heading: double (nullable = true)
 |-- surface: string (nullable = true)
 |-- has_lighting: boolean (nullable = true)
 |-- is_closed: boolean (nullable = true)
 |-- length_feet: double (nullable = true)
 |-- width_feet: double (nullable = true)
 |-- displaced_threshold_feet: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [28]:
write_delta_table_with_scd(spark, dim_runways, "silver", "dim_runways", business_keys=["airport_sk", "true_heading", "latitude", "longitude"], surrogate_key="runway_sk")

root
 |-- airport_sk: long (nullable = true)
 |-- runway_name: string (nullable = true)
 |-- true_heading: double (nullable = true)
 |-- surface: string (nullable = true)
 |-- has_lighting: boolean (nullable = true)
 |-- is_closed: boolean (nullable = true)
 |-- length_feet: double (nullable = true)
 |-- width_feet: double (nullable = true)
 |-- displaced_threshold_feet: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- row_hash: string (nullable = true)

root
 |-- runway_sk: long (nullable = true)
 |-- airport_sk: long (nullable = true)
 |-- runway_name: string (nullable = true)
 |-- true_heading: double (nullable = true)
 |-- surface: string (nullable = true)
 |-- has_lighting: boolean (nullable = true)
 |-- is_closed: boolean (nullable = true)
 |-- length_feet: double (nullable = true)
 |-- width_feet: double (nullable = true)
 |-- displaced_threshold_feet: double (nullable = true)
 |-- latitude: double (nullable = true)
 

In [15]:
silver_dim_runways = read_df(spark, "silver", "dim_runways")
silver_dim_runways.show(5)

Successfully read silver.dim_runways
+--------------------+--------------------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+---------+---------+--------------------+------------------+
|           runway_sk|          airport_sk|runway_name|true_heading|surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet| latitude|longitude|effective_start_date|effective_end_date|
+--------------------+--------------------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+---------+---------+--------------------+------------------+
| 3776516148712988071| 7061772760354646290|        17L|       169.0|Asphalt|        true|    false|    12877.0|     197.0|                 3097.11| 45.65034| 8.727936|2025-10-10 09:19:...|              NULL|
| 8233962487606072415| 7061772760354646290|        17R|       169.1|Asphalt|        true|    false|    12877.0|     197.0|         

In [10]:
fct_flights.show(5, truncate=False)

25/10/10 09:57:21 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------------------------------------------------------+--------------------+--------------------+--------------------+--------------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+-----------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+-------------+------------+--------+----------------+----------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_composite_pk                                                 |departure_airport_sk|arrival_airport_sk  |airline_sk          |aircraft_sk         |flight_number|flight_callsign|flight_status|codeshare_status|is_cargo|dep_s

In [13]:
silver_dim_runways.filter(col("runway_name") == "UNKNOWN").show(5  )

+--------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------+------------------+
|           runway_sk|airport_sk|runway_name|true_heading|surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet|latitude|longitude|effective_start_date|effective_end_date|
+--------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------+------------------+
|-4370665737390788322|        -1|    UNKNOWN|         0.0|Unknown|       false|    false|        0.0|       0.0|                     0.0|     0.0|      0.0|2025-10-10 09:31:...|              NULL|
+--------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------+------------------+



In [17]:
from pyspark.sql.functions import lit, coalesce, col

dep_runway_dim = silver_dim_runways.alias("dep")
arr_runway_dim = silver_dim_runways.alias("arr")

fct_flights = enriched_union_flights.join(
    dep_runway_dim,
    on=[enriched_union_flights["departure_runway"] == dep_runway_dim["runway_name"], enriched_union_flights["departure_airport_sk"] == dep_runway_dim["airport_sk"]],
    how="left"
).join(
    arr_runway_dim,
    on=[enriched_union_flights["arrival_runway"] == arr_runway_dim["runway_name"], enriched_union_flights["arrival_airport_sk"] == arr_runway_dim["airport_sk"]],
    how="left"
)

unknown_runway_sk = silver_dim_runways.filter(col("airport_sk") == -1).select("runway_sk").collect()[0]["runway_sk"]

fct_flights = fct_flights.withColumn(
    "departure_runway_sk",
    coalesce(col("dep.runway_sk"), lit(unknown_runway_sk))
).withColumn(
    "arrival_runway_sk",
    coalesce(col("arr.runway_sk"), lit(unknown_runway_sk))
)

fct_flights = fct_flights.fillna(unknown_runway_sk, subset=["departure_runway_sk", "arrival_runway_sk"])

from utils.expression_utils import get_select_expressions
fct_flights_select_exprs = get_select_expressions("silver", "fct_flights")

fct_flights = fct_flights.select(*fct_flights_select_exprs)
fct_flights.show(5, truncate=False)

+--------------------------------------------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+-----------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+-------------+------------+--------+----------------+----------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_composite_pk                                                 |departure_airport_sk|arrival_airport_sk  |airline_sk          |aircraft_sk         |departure_runway_sk |arrival_runw

In [20]:
from process.utils import write_delta_table

write_delta_table(spark, fct_flights, "silver", "fct_flights", partition_cols=["ingestion_hour"])

25/10/10 13:39:15 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`silver`.`fct_flights` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
25/10/10 13:39:15 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
25/10/10 13:39:15 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
25/10/10 13:39:15 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/10/10 13:39:15 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [80]:
dim_runways.printSchema()

root
 |-- airport_sk: long (nullable = false)
 |-- runway_name: string (nullable = true)
 |-- true_heading: double (nullable = true)
 |-- surface: string (nullable = true)
 |-- has_lighting: boolean (nullable = true)
 |-- is_closed: boolean (nullable = true)
 |-- length_feet: double (nullable = true)
 |-- width_feet: double (nullable = true)
 |-- displaced_threshold_feet: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [25]:
fct_flights.filter(sf.length(col("arrival_airport_iata")) > 3).select(sf.count_distinct(col("arrival_airport_iata"))).show()
# fct_flights.filter(sf.length(col("arrival_airport_iata")) > 3).show()

+------------------------------------+
|count(DISTINCT arrival_airport_iata)|
+------------------------------------+
|                                  31|
+------------------------------------+



In [10]:
departure_flights.printSchema()

root
 |-- flight_pk: string (nullable = true)
 |-- flight_composite_pk: string (nullable = false)
 |-- departure_airport_iata: string (nullable = true)
 |-- arrival_airport_iata: string (nullable = true)
 |-- airline_iata: string (nullable = true)
 |-- aircraft_reg: string (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- flight_callsign: string (nullable = true)
 |-- flight_status: string (nullable = true)
 |-- codeshare_status: string (nullable = true)
 |-- is_cargo: boolean (nullable = true)
 |-- dep_scheduled_at_utc: timestamp (nullable = true)
 |-- dep_revised_at_utc: timestamp (nullable = true)
 |-- dep_predicted_at_utc: timestamp (nullable = true)
 |-- dep_runway_at_utc: timestamp (nullable = true)
 |-- dep_scheduled_at_local_ntz: string (nullable = true)
 |-- dep_revised_at_local_ntz: string (nullable = true)
 |-- dep_predicted_at_local_ntz: string (nullable = true)
 |-- dep_runway_at_local_ntz: string (nullable = true)
 |-- dep_local_timezone: string (nullabl

In [26]:
from pyspark.sql.functions import explode

arrival_exploded_df = (
    bronze_flights.select(
        explode("arrivals").alias("data"),
        "airport",
        "ingestion_hour",
    )
)

arrival_exploded_df.count()

27350

In [4]:
from utils.io_utils import read_df

fct_flights = read_df(spark, "silver", "fct_flights")
fct_flights.show(5)

25/10/07 16:24:27 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/10/07 16:24:27 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/10/07 16:24:29 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/10/07 16:24:29 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore dottier@127.0.1.1
25/10/07 16:24:29 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


Successfully read silver.fct_flights


25/10/07 16:24:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+------------------+----------+--------------------+-------------+---------------+-----------------+----------------+--------+--------------------+-------------------+--------------------+-----------------+----------------------------------+--------------------------------+----------------------------------+-------------------------------+------------------+--------------------+-------------------+--------------------+-----------------+--------------------------------+------------------------------+--------------------------------+-----------------------------+------------------+------------+--------+------------+----------------+-------------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|           flight_pk| flight_composite_pk|departure_airport_pk|arrival_airport_pk|airline_pk|         aircraft_pk|flight_number|flight_callsign|    flight_status